# Purpose
Identify data gaps in the ownership data.

## Overview
Review linkages which are apparent in the ownership dataset for:
- Main Line Health
- Jefferson Health
- Penn Medicine

## Key Questions
### Are the `associate_id` drawing a link to effectively characterize the "health system"?
If you link by individuals yes... For instance, CEO of Main Line Health is uniquely identified in both Main Line HHA and Main Line Hospitals. The organization owner is not uniquely identified, the link is unclear.

In [1]:
import os
os.chdir("..")
os.chdir("..")
os.getcwd()

'/home/dr00b/code/slothful_sleuth'

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///data/raw/cms_data_api_raw.db
%config SqlMagic.displaylimit = 100

Connecting to 'sqlite:///data/raw/cms_data_api_raw.db'

%md

### Main Line Health
The link between the health system and the HHA is not apparent at the organization level. But same individual associates on the board.
- MAIN LINE HOSPITALS, INC.
- MAIN LINE DIVERSIFIED SERVICES

In [13]:
%%sql
SELECT * FROM hospital_enrollments e
INNER JOIN hospital_all_owners o
ON e.associate_id = o.associate_id
WHERE e.state = 'PA'
AND e.organization_name = 'MAIN LINE HOSPITALS, INC.'
GROUP BY e.ccn
LIMIT 100

Running query in 'sqlite:///data/raw/cms_data_api_raw.db'

enrollment_id,enrollment_state,provider_type_code,provider_type_text,npi,multiple_npi_flag,ccn,associate_id,organization_name,doing_business_as_name,incorporation_date,incorporation_state,organization_type_structure,organization_other_type_text,proprietary_nonprofit,address_line_1,address_line_2,city,state,zip_code,practice_location_type,location_other_type_text,subgroup_-_general,subgroup_-_acute_care,subgroup_-_alcohol_drug,subgroup_-_childrens,subgroup_-_long-term,subgroup_-_psychiatric,subgroup_-_rehabilitation,subgroup_-_short-term,subgroup_-_swing-bed_approved,subgroup_-_psychiatric_unit,subgroup_-_rehabilitation_unit,subgroup_-_specialty_hospital,subgroup_-_other,subgroup_-_other_text,reh_conversion_flag,reh_conversion_date,cah_or_hospital_ccn,distro_access_url,distro_title,distro_modified,_load_ts,enrollment_id_1,associate_id_1,organization_name_1,associate_id_-_owner,type_-_owner,role_code_-_owner,role_text_-_owner,association_date_-_owner,first_name_-_owner,middle_name_-_owner,last_name_-_owner,title_-_owner,organization_name_-_owner,doing_business_as_name_-_owner,address_line_1_-_owner,address_line_2_-_owner,city_-_owner,state_-_owner,zip_code_-_owner,percentage_ownership,created_for_acquisition_-_owner,corporation_-_owner,llc_-_owner,medical_provider_supplier_-_owner,management_services_company_-_owner,medical_staffing_company_-_owner,holding_company_-_owner,investment_firm_-_owner,financial_institution_-_owner,consulting_firm_-_owner,for_profit_-_owner,non_profit_-_owner,other_type_-_owner,other_type_text_-_owner,distro_access_url_1,distro_title_1,distro_modified_1,_load_ts_1
O20070830000285,PA,00-09,PART A PROVIDER - HOSPITAL,1508971441,N,390139,0648168955,"MAIN LINE HOSPITALS, INC.",BRYN MAWR HOSPITAL,1996-07-01,PA,CORPORATION,None,N,130 S BRYN MAWR AVE,None,BRYN MAWR,PA,190103121,MAIN/PRIMARY HOSPITAL LOCATION,None,Y,Y,N,N,N,N,N,N,N,N,N,N,N,None,N,None,None,https://data.cms.gov/data-api/v1/dataset/f6f6505c-e8b0-4d57-b258-e2b94133aaf2/data,Hospital Enrollments : 2024-04-01,2024-04-15,2024-04-28 15:52:00.332467,O20070601000473,0648168955,"MAIN LINE HOSPITALS, INC.",0446636591,I,41,DIRECTOR,2022-07-01,BRADFORD,S,KOLES,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,https://data.cms.gov/data-api/v1/dataset/029c119f-f79c-49be-9100-344d31d10344/data,Hospital All Owners : 2024-04-01,2024-04-15,2024-04-28 15:46:52.068356
O20070601000473,PA,00-09,PART A PROVIDER - HOSPITAL,1215042114,N,390153,0648168955,"MAIN LINE HOSPITALS, INC.",PAOLI HOSPITAL,1996-07-01,PA,CORPORATION,None,N,255 W LANCASTER AVE,PAOLI HOSPITAL,PAOLI,PA,193011792,OTHER HOSPITAL PRACTICE LOCATION,None,Y,Y,N,N,N,N,N,N,N,N,N,N,N,None,N,None,None,https://data.cms.gov/data-api/v1/dataset/f6f6505c-e8b0-4d57-b258-e2b94133aaf2/data,Hospital Enrollments : 2024-04-01,2024-04-15,2024-04-28 15:52:00.332467,O20070601000473,0648168955,"MAIN LINE HOSPITALS, INC.",0446636591,I,41,DIRECTOR,2022-07-01,BRADFORD,S,KOLES,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,https://data.cms.gov/data-api/v1/dataset/029c119f-f79c-49be-9100-344d31d10344/data,Hospital All Owners : 2024-04-01,2024-04-15,2024-04-28 15:46:52.068356
O20071019000501,PA,00-09,PART A PROVIDER - HOSPITAL,1245335629,N,390195,0648168955,"MAIN LINE HOSPITALS, INC.",LANKENAU MEDICAL CENTER,1996-07-01,PA,CORPORATION,None,N,100 E LANCASTER AVE,None,WYNNEWOOD,PA,190963450,OTHER HOSPITAL PRACTICE LOCATION,None,Y,Y,N,N,N,N,N,N,N,N,N,N,Y,ORGAN TRANSPLANT PROGRAM,N,None,None,https://data.cms.gov/data-api/v1/dataset/f6f6505c-e8b0-4d57-b258-e2b94133aaf2/data,Hospital Enrollments : 2024-04-01,2024-04-15,2024-04-28 15:52:00.332467,O20070601000473,0648168955,"MAIN LINE HOSPITALS, INC.",0446636591,I,41,DIRECTOR,2022-07-01,BRADFORD,S,KOLES,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,https://data.cms.gov/data-api/v1/dataset/

In [22]:
%%sql
SELECT DISTINCT `associate_id_owner` FROM hha_all_owners
WHERE `first_name_owner` = 'JOHN'
AND `last_name_owner` = 'LYNCH'
-- 6204182892 is a different John Lynch (middle name disambiguates)

Running query in 'sqlite:///data/raw/cms_data_api_raw.db'

associate_id_-_owner
2567469752
6204182892


In [17]:
%%sql
SELECT DISTINCT `associate_id_owner` FROM hospital_all_owners o
WHERE organization_name = 'MAIN LINE HOSPITALS, INC.'
AND `first_name_owner` = 'JOHN'
AND `last_name_owner` = 'LYNCH'

Running query in 'sqlite:///data/raw/cms_data_api_raw.db'

associate_id_-_owner
2567469752


In [26]:
%%sql
SELECT *  FROM hha_all_owners
WHERE organization_name = 'MAIN LINE HEALTH HOMECARE AND HOSPICE'
AND `type_owner` = 'O'

Running query in 'sqlite:///data/raw/cms_data_api_raw.db'

enrollment_id,associate_id,organization_name,associate_id_-_owner,type_-_owner,role_code_-_owner,role_text_-_owner,association_date_-_owner,first_name_-_owner,middle_name_-_owner,last_name_-_owner,title_-_owner,organization_name_-_owner,doing_business_as_name_-_owner,address_line_1_-_owner,address_line_2_-_owner,city_-_owner,state_-_owner,zip_code_-_owner,percentage_ownership,created_for_acquisition_-_owner,corporation_-_owner,llc_-_owner,medical_provider_supplier_-_owner,management_services_company_-_owner,medical_staffing_company_-_owner,holding_company_-_owner,investment_firm_-_owner,financial_institution_-_owner,consulting_firm_-_owner,for_profit_-_owner,non_profit_-_owner,other_type_-_owner,other_type_text_-_owner,distro_access_url,distro_title,distro_modified,_load_ts
O20020925000029,1052228780,MAIN LINE HEALTH HOMECARE AND HOSPICE,7618866450,O,34,5% OR GREATER DIRECT OWNERSHIP INTEREST,1996-04-19,None,None,None,None,MAIN LINE DIVERSIFIED SERVICES,None,240 N RADNOR CHESTER RD,STE 200,RADNOR,PA,190872126,100,N,N,N,N,N,N,N,N,N,N,N,Y,N,None,https://data.cms.gov/data-api/v1/dataset/fc009b2d-7846-44b1-b4a1-692f0c143879/data,Home Health Agency All Owners : 2024-04-01,2024-04-15,2024-04-28 15:47:16.417379
O20020925000029,1052228780,MAIN LINE HEALTH HOMECARE AND HOSPICE,7618866450,O,34,5% OR GREATER DIRECT OWNERSHIP INTEREST,1996-04-19,None,None,None,None,MAIN LINE DIVERSIFIED SERVICES,None,240 N RADNOR CHESTER RD,STE 200,RADNOR,PA,190872126,100,N,N,N,N,N,N,N,N,N,N,N,Y,N,None,https://data.cms.gov/data-api/v1/dataset/fc009b2d-7846-44b1-b4a1-692f0c143879/data,Home Health Agency All Owners : 2024-04-01,2024-04-15,2024-04-28 15:50:24.323673
